In [2]:

# Recursive Multi-Scale Zombie Outbreak Simulation
# Inspired by "When Zombies Attack!" by Munz et al.
# Implements a fractal-like recursive structure of nested automata

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import animation
from matplotlib import rcParams
from matplotlib.patches import Rectangle
from IPython.display import HTML, Video
import numba
import time
import os
from tqdm import tqdm

# Increase animation size limit to avoid frame dropping
rcParams['animation.embed_limit'] = 100  # 100 MB

# Constants for simulation states
EMPTY = 0
SUSCEPTIBLE = 1
INFECTED = 2
ZOMBIE = 3
REMOVED = 4

# Define recursive structure
RECURSION_DEPTH = 2  # How many levels of recursion (1=cities, 2=regions of cities, etc.)
CELLS_PER_DIMENSION = 3  # How many cells per dimension at each level (3×3 grid)

# Calculate total grid size
BASE_GRID_SIZE = 50  # Size of the smallest grid unit
TOTAL_GRID_SIZE = BASE_GRID_SIZE * (CELLS_PER_DIMENSION ** RECURSION_DEPTH)

print(f"Initializing simulation with {RECURSION_DEPTH} recursion levels")
print(f"Total grid dimensions: {TOTAL_GRID_SIZE}×{TOTAL_GRID_SIZE}")

# ========================== PARAMETERS ==========================

# City/region names at different scales (add more if needed)
AREA_NAMES = {
    0: {  # Level 0 (smallest units)
        'Center': (1, 1),
        'North': (0, 1),
        'South': (2, 1),
        'East': (1, 2),
        'West': (1, 0),
        'Northeast': (0, 2),
        'Northwest': (0, 0),
        'Southeast': (2, 2),
        'Southwest': (2, 0),
    },
    1: {  # Level 1 (3×3 grids of level 0)
        'Arizona': (1, 1),
        'California': (1, 0),
        'New Mexico': (1, 2),
        'Nevada': (0, 0),
        'Utah': (0, 1),
        'Colorado': (0, 2),
        'Sonora': (2, 0),
        'Chihuahua': (2, 1),
        'Texas': (2, 2),
    }
}

# City configurations (baseline parameters for each city)
CITY_PARAMS = {
    'Center': {
        'beta': 0.5,      # Transmission probability
        'rho': 0.3,       # Rate at which infected become zombies
        'alpha': 0.1,     # Rate at which zombies are destroyed
        'zeta': 0.05,     # Rate at which removed resurrect as zombies
        'delta': 0.01,    # Natural death rate
        'pi': 0.02,       # Birth rate (new susceptibles)
        'density': 0.4,   # Initial population density
    },
    'North': {
        'beta': 0.45,     # Lower transmission in less dense area
        'rho': 0.25,      # Slower zombification (cooler climate)
        'alpha': 0.15,    # Better defense (isolated community)
        'zeta': 0.04,
        'delta': 0.01,
        'pi': 0.015,
        'density': 0.25,  # Lower population density
    },
    'South': {
        'beta': 0.45,
        'rho': 0.3,
        'alpha': 0.1,
        'zeta': 0.05,
        'delta': 0.01,
        'pi': 0.015,
        'density': 0.25,
    },
    'East': {
        'beta': 0.5,
        'rho': 0.3,
        'alpha': 0.1,
        'zeta': 0.05,
        'delta': 0.01,
        'pi': 0.02,
        'density': 0.35,
    },
    'West': {
        'beta': 0.5,
        'rho': 0.35,      # Faster zombification (hot climate)
        'alpha': 0.1,
        'zeta': 0.06,     # Faster resurrection (hot climate)
        'delta': 0.01,
        'pi': 0.02,
        'density': 0.3,
    },
    'Northeast': {
        'beta': 0.4,
        'rho': 0.25,
        'alpha': 0.2,     # Better defense (prepared community)
        'zeta': 0.03,
        'delta': 0.01,
        'pi': 0.01,
        'density': 0.2,   # Low population density
    },
    'Northwest': {
        'beta': 0.55,     # Higher transmission in dense urban area
        'rho': 0.3,
        'alpha': 0.08,    # Slightly less effective defense
        'zeta': 0.05,
        'delta': 0.01,
        'pi': 0.025,      # Slightly higher birth rate
        'density': 0.5,   # Higher population density
    },
    'Southeast': {
        'beta': 0.45,
        'rho': 0.3,
        'alpha': 0.1,
        'zeta': 0.05,
        'delta': 0.01,
        'pi': 0.015,
        'density': 0.25,
    },
    'Southwest': {
        'beta': 0.45,
        'rho': 0.3,
        'alpha': 0.12,    # Better defense (military presence)
        'zeta': 0.05,
        'delta': 0.01,
        'pi': 0.02,
        'density': 0.3,
    }
}

# Define color scheme for visualization
def create_colormap():
    """Create a beautiful colormap for visualization"""
    colors = [
        (0.05, 0.05, 0.15),        # Empty (dark blue)
        (0.0, 0.8, 0.5),           # Susceptible (teal)
        (1.0, 0.85, 0.2),          # Infected (yellow-gold)
        (0.95, 0.15, 0.15),        # Zombie (red)
        (0.6, 0.3, 0.6)            # Removed (purple)
    ]
    return ListedColormap(colors)

# ========================== UTILITY FUNCTIONS ==========================

def get_area_name(level, x, y):
    """Get the name of an area at a specific level and position"""
    for name, pos in AREA_NAMES[level].items():
        if pos == (y, x):  # Note: positions are (y, x) not (x, y)
            return name
    return f"Area_{y}_{x}"

def get_params_for_position(grid_y, grid_x, base_size, recursion_level=0):
    """
    Get parameters for a specific position in the grid based on its hierarchy
    This recursively determines which parameters apply at each level
    """
    # Calculate which segment we're in at this level
    segment_size = base_size * (CELLS_PER_DIMENSION ** recursion_level)
    segment_y = grid_y // segment_size
    segment_x = grid_x // segment_size
    
    # Get area name for this position at this level
    area_name = get_area_name(recursion_level, segment_x, segment_y)
    
    # Start with default parameters
    if area_name in CITY_PARAMS:
        params = CITY_PARAMS[area_name].copy()
    else:
        params = CITY_PARAMS['Center'].copy()  # Default to center if not found
    
    # Recursively modify parameters based on higher-level regions (if applicable)
    if recursion_level < RECURSION_DEPTH - 1:
        parent_params = get_params_for_position(
            grid_y, grid_x, base_size, recursion_level + 1
        )
        
        # Higher-level regions influence lower-level parameters
        # Here we apply a weighted modification based on the higher level
        influence_weight = 0.2  # How much parent regions influence child regions
        params['beta'] = (1-influence_weight) * params['beta'] + influence_weight * parent_params['beta']
        params['rho'] = (1-influence_weight) * params['rho'] + influence_weight * parent_params['rho']
        params['alpha'] = (1-influence_weight) * params['alpha'] + influence_weight * parent_params['alpha']
        params['zeta'] = (1-influence_weight) * params['zeta'] + influence_weight * parent_params['zeta']
        params['density'] = (1-influence_weight) * params['density'] + influence_weight * parent_params['density']
        
    return params

# ========================== GRID INITIALIZATION ==========================

def initialize_recursive_grid():
    """Initialize the grid with nested structure and parameters"""
    # Main grid holds all cells across all regions
    grid = np.zeros((TOTAL_GRID_SIZE, TOTAL_GRID_SIZE), dtype=np.int8)
    
    # Parameter grid stores cell-specific parameters
    param_grid = np.zeros((TOTAL_GRID_SIZE, TOTAL_GRID_SIZE, 6), dtype=np.float32)
    
    # Label grid for visualization and tracking
    label_grid = np.zeros((TOTAL_GRID_SIZE, TOTAL_GRID_SIZE), dtype=np.int8)
    
    # Fill grids with appropriate values
    for y in range(TOTAL_GRID_SIZE):
        for x in range(TOTAL_GRID_SIZE):
            # Get parameters for this cell based on its position in the hierarchy
            params = get_params_for_position(y, x, BASE_GRID_SIZE)
            
            # Set parameters for this cell
            param_grid[y, x, 0] = params['beta']    # Transmission rate
            param_grid[y, x, 1] = params['rho']     # Zombification rate
            param_grid[y, x, 2] = params['alpha']   # Zombie destruction rate
            param_grid[y, x, 3] = params['zeta']    # Resurrection rate
            param_grid[y, x, 4] = params['delta']   # Natural death rate
            param_grid[y, x, 5] = params['pi']      # Birth rate
            
            # Populate with susceptible based on density
            if np.random.random() < params['density']:
                grid[y, x] = SUSCEPTIBLE
                
            # Set label based on hierarchical position
            for level in range(RECURSION_DEPTH):
                segment_size = BASE_GRID_SIZE * (CELLS_PER_DIMENSION ** level)
                segment_y = y // segment_size % CELLS_PER_DIMENSION
                segment_x = x // segment_size % CELLS_PER_DIMENSION
                label_grid[y, x] += (segment_y * CELLS_PER_DIMENSION + segment_x + 1) * (10 ** level)
    
    # Create hierarchy information for tracking and visualization
    hierarchy_info = {}
    for level in range(RECURSION_DEPTH):
        segment_size = BASE_GRID_SIZE * (CELLS_PER_DIMENSION ** level)
        hierarchy_info[level] = {
            'size': segment_size,
            'areas': {}
        }
        for y in range(CELLS_PER_DIMENSION ** (RECURSION_DEPTH - level)):
            for x in range(CELLS_PER_DIMENSION ** (RECURSION_DEPTH - level)):
                area_y = y // (CELLS_PER_DIMENSION ** (RECURSION_DEPTH - level - 1))
                area_x = x // (CELLS_PER_DIMENSION ** (RECURSION_DEPTH - level - 1))
                area_name = get_area_name(level, area_x, area_y)
                y_start = y * segment_size
                y_end = (y + 1) * segment_size
                x_start = x * segment_size
                x_end = (x + 1) * segment_size
                
                hierarchy_info[level]['areas'][(y, x)] = {
                    'name': area_name,
                    'bounds': (y_start, y_end, x_start, x_end)
                }
    
    # Add initial zombie outbreak
    # Start in the center city (Tucson in our Arizona model)
    center_level = RECURSION_DEPTH - 1  # Highest level
    center_pos = AREA_NAMES[center_level]['Arizona']
    segment_size = BASE_GRID_SIZE * (CELLS_PER_DIMENSION ** (RECURSION_DEPTH - 1))
    center_y = center_pos[0] * segment_size + segment_size // 2
    center_x = center_pos[1] * segment_size + segment_size // 2
    
    # Place patient zero and surrounding infections
    for i in range(-2, 3):
        for j in range(-2, 3):
            if np.random.random() < 0.7:  # 70% chance for each cell
                grid[center_y + i, center_x + j] = ZOMBIE
    
    return grid, param_grid, label_grid, hierarchy_info

# ========================== CORE SIMULATION LOGIC ==========================

# Pre-compute neighbor offsets for faster access
NEIGHBOR_OFFSETS = np.array([
    (-1, -1), (-1, 0), (-1, 1),
    (0, -1),           (0, 1),
    (1, -1),  (1, 0),  (1, 1)
])

# Define core update function with JIT compilation for speed
@numba.njit
def update_recursive_grid(grid, param_grid, rng_seed=None):
    """Update the grid according to zombie infection rules with area-specific parameters"""
    if rng_seed is not None:
        np.random.seed(rng_seed)
    
    rows, cols = grid.shape
    new_grid = grid.copy()
    
    # Create random numbers in advance
    random_vals = np.random.random((rows, cols, 4))
    
    # First pass: Apply core transition rules
    for i in range(rows):
        for j in range(cols):
            state = grid[i, j]
            
            # Get parameters for this cell's location
            beta = param_grid[i, j, 0]    # Transmission rate
            rho = param_grid[i, j, 1]     # Zombification rate
            alpha = param_grid[i, j, 2]   # Zombie destruction rate
            zeta = param_grid[i, j, 3]    # Resurrection rate
            delta = param_grid[i, j, 4]   # Natural death rate
            pi = param_grid[i, j, 5]      # Birth rate
            
            # Count neighbors
            zombie_neighbors = 0
            susceptible_neighbors = 0
            
            for di, dj in NEIGHBOR_OFFSETS:
                ni, nj = (i + di) % rows, (j + dj) % cols
                neighbor_state = grid[ni, nj]
                if neighbor_state == ZOMBIE:
                    zombie_neighbors += 1
                elif neighbor_state == SUSCEPTIBLE:
                    susceptible_neighbors += 1
            
            # Use pre-computed random values
            r1, r2, r3, r4 = random_vals[i, j]
            
            # Apply transition rules
            if state == EMPTY:
                if r1 < pi:
                    new_grid[i, j] = SUSCEPTIBLE
            
            elif state == SUSCEPTIBLE:
                # Infection from zombie neighbors
                if zombie_neighbors > 0 and r1 < beta * zombie_neighbors / 8:
                    new_grid[i, j] = INFECTED
                # Natural death
                elif r2 < delta:
                    new_grid[i, j] = REMOVED
            
            elif state == INFECTED:
                if r1 < rho:
                    new_grid[i, j] = ZOMBIE
                elif r2 < delta:
                    new_grid[i, j] = REMOVED
            
            elif state == ZOMBIE:
                # Destruction when encountering susceptibles
                if susceptible_neighbors > 0 and r1 < alpha * susceptible_neighbors / 8:
                    new_grid[i, j] = REMOVED
                # Zombies move randomly
                elif r2 < 0.5:  # 50% chance to move
                    # Choose direction based on r3
                    direction = int(r3 * 8)
                    di, dj = NEIGHBOR_OFFSETS[direction]
                    ni, nj = (i + di) % rows, (j + dj) % cols
                    
                    if grid[ni, nj] == EMPTY:
                        new_grid[i, j] = EMPTY
                        new_grid[ni, nj] = ZOMBIE
            
            elif state == REMOVED:
                if r1 < zeta:
                    new_grid[i, j] = ZOMBIE
    
    # Second pass: Inter-region zombie migration
    # This simulates both short-distance and long-distance travel
    for i in range(rows):
        for j in range(cols):
            if new_grid[i, j] == ZOMBIE:
                # Regular neighborhood migration - 5% chance
                if np.random.random() < 0.05:
                    # Choose a direction
                    angle = np.random.random() * 2 * np.pi
                    distance = (np.random.random() * 10 + 5)  # 5-15 cells
                    
                    # Calculate new position
                    new_i = int(i + np.sin(angle) * distance) % rows
                    new_j = int(j + np.cos(angle) * distance) % cols
                    
                    if new_grid[new_i, new_j] == EMPTY:
                        new_grid[i, j] = EMPTY
                        new_grid[new_i, new_j] = ZOMBIE
                
                # Long-distance migration - 1% chance
                elif np.random.random() < 0.01:
                    # Choose a random distant location
                    new_i = np.random.randint(0, rows)
                    new_j = np.random.randint(0, cols)
                    
                    if new_grid[new_i, new_j] == EMPTY:
                        new_grid[i, j] = EMPTY
                        new_grid[new_i, new_j] = ZOMBIE
    
    return new_grid

# ========================== STATISTICS AND TRACKING ==========================

def count_population_by_area(grid, hierarchy_info):
    """Count populations across different hierarchical levels"""
    hierarchy_stats = {}
    
    # Get overall stats first
    overall_stats = count_overall_population(grid)
    hierarchy_stats['overall'] = overall_stats
    
    # Get stats for each level in the hierarchy
    for level, level_info in hierarchy_info.items():
        level_stats = {}
        for coords, area_info in level_info['areas'].items():
            area_name = area_info['name']
            y_start, y_end, x_start, x_end = area_info['bounds']
            
            # Extract the grid section for this area
            area_grid = grid[y_start:y_end, x_start:x_end]
            
            # Count states
            unique, counts = np.unique(area_grid, return_counts=True)
            population = {state: 0 for state in range(5)}
            
            for state, count in zip(unique, counts):
                population[state] = count
            
            level_stats[area_name] = population
        
        hierarchy_stats[level] = level_stats
    
    return hierarchy_stats

def count_overall_population(grid):
    """Count the total number of individuals in each state"""
    unique, counts = np.unique(grid, return_counts=True)
    population = {state: 0 for state in range(5)}
    
    for state, count in zip(unique, counts):
        population[state] = count
    
    return population

def calculate_infection_metrics(stats):
    """Calculate epidemic metrics from population statistics"""
    metrics = {}
    
    # Overall infection rate
    total_population = sum(stats['overall'].values())
    infected_count = stats['overall'].get(INFECTED, 0)
    zombie_count = stats['overall'].get(ZOMBIE, 0)
    susceptible_count = stats['overall'].get(SUSCEPTIBLE, 0)
    removed_count = stats['overall'].get(REMOVED, 0)
    
    metrics['infected_rate'] = (infected_count + zombie_count) / total_population if total_population > 0 else 0
    metrics['active_rate'] = (infected_count + zombie_count + susceptible_count) / total_population if total_population > 0 else 0
    metrics['mortality_rate'] = removed_count / total_population if total_population > 0 else 0
    
    # Area-specific calculations
    metrics['hotspot_regions'] = []
    
    # Calculate for each region
    for level in range(RECURSION_DEPTH):
        if level in stats:
            level_metrics = {}
            for area_name, area_stats in stats[level].items():
                area_total = sum(area_stats.values())
                if area_total == 0:
                    continue
                    
                infected = area_stats.get(INFECTED, 0)
                zombie = area_stats.get(ZOMBIE, 0)
                area_infection_rate = (infected + zombie) / area_total
                
                if area_infection_rate > 0.3:  # Threshold for a "hotspot"
                    metrics['hotspot_regions'].append((level, area_name, area_infection_rate))
                
                level_metrics[area_name] = area_infection_rate
            
            metrics[f'level_{level}_infection'] = level_metrics
    
    return metrics

# ========================== VISUALIZATION ==========================

def get_boundary_coordinates(level, hierarchy_info):
    """Get coordinates for drawing boundaries at specified level"""
    boundaries = []
    segment_size = hierarchy_info[level]['size']
    
    for y in range(0, TOTAL_GRID_SIZE, segment_size):
        boundaries.append(((0, y), (TOTAL_GRID_SIZE, y)))
        
    for x in range(0, TOTAL_GRID_SIZE, segment_size):
        boundaries.append(((x, 0), (x, TOTAL_GRID_SIZE)))
        
    return boundaries

def visualize_metrics(metrics, ax):
    """Visualize metrics on a separate axis"""
    # Clear previous text
    ax.clear()
    ax.axis('off')
    
    # Basic stats
    text = [
        f"Infection Rate: {metrics['infected_rate']*100:.1f}%",
        f"Active Population: {metrics['active_rate']*100:.1f}%",
        f"Mortality Rate: {metrics['mortality_rate']*100:.1f}%",
        "\nHotspot Regions:"
    ]
    
    # Add hotspot info
    for level, area, rate in metrics['hotspot_regions'][:5]:  # Limit to top 5
        text.append(f"- {area} (L{level}): {rate*100:.1f}%")
    
    # Add text to axis
    ax.text(0.05, 0.95, '\n'.join(text), 
            transform=ax.transAxes, fontsize=9,
            verticalalignment='top', bbox=dict(boxstyle='round', 
                                               facecolor='white', 
                                               alpha=0.8))

def visualize_hierarchy(hierarchy_info, frame_stats, ax):
    """Visualize the hierarchical structure as a nested grid"""
    # Clear previous plots
    ax.clear()
    ax.set_xlim(0, CELLS_PER_DIMENSION)
    ax.set_ylim(0, CELLS_PER_DIMENSION)
    ax.axis('off')
    
    # Only visualize the top level of the hierarchy
    level = RECURSION_DEPTH - 1
    level_stats = frame_stats.get(level, {})
    
    # Create color mapping based on infection rates
    cmap = plt.cm.RdYlGn_r  # Red (high infection) to Green (low infection)
    
    # Draw each area as a colored rectangle
    for coords, area_info in hierarchy_info[level]['areas'].items():
        y, x = coords
        area_name = area_info['name']
        
        # Get infection rate for this area
        area_stats = level_stats.get(area_name, {})
        area_total = sum(area_stats.values()) if area_stats else 0
        infected = area_stats.get(INFECTED, 0) if area_stats else 0
        zombie = area_stats.get(ZOMBIE, 0) if area_stats else 0
        infection_rate = (infected + zombie) / area_total if area_total > 0 else 0
        
        # Create rectangle
        rect = Rectangle((x, y), 1, 1, 
                        facecolor=cmap(infection_rate), 
                        edgecolor='black', 
                        alpha=0.7)
        ax.add_patch(rect)
        
        # Add text label
        ax.text(x + 0.5, y + 0.5, area_name,
                ha='center', va='center', fontsize=8)

# ========================== SIMULATION RUNNER ==========================

def run_recursive_simulation(num_frames=150, save_interval=2, interval=100, save_video=False):
    """Run the full recursive simulation"""
    start_time = time.time()
    print("Initializing simulation...")
    grid, param_grid, label_grid, hierarchy_info = initialize_recursive_grid()
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(16, 10))
    grid_ax = plt.subplot2grid((2, 3), (0, 0), colspan=2, rowspan=2)
    metrics_ax = plt.subplot2grid((2, 3), (0, 2))
    hierarchy_ax = plt.subplot2grid((2, 3), (1, 2))
    
    # Get colormap
    cmap = create_colormap()
    
    # Initial plot
    grid_img = grid_ax.imshow(grid, cmap=cmap, vmin=0, vmax=4)
    grid_ax.set_title('Recursive Multi-Scale Zombie Outbreak Simulation', fontsize=14)
    grid_ax.axis('off')
    
    # Add level 0 grid lines (city boundaries)
    for ((x1, y1), (x2, y2)) in get_boundary_coordinates(0, hierarchy_info):
        grid_ax.plot([x1, x2], [y1, y2], color='white', alpha=0.3, linewidth=0.5)
        
    # Add level 1+ grid lines (region boundaries)
    for level in range(1, RECURSION_DEPTH):
        for ((x1, y1), (x2, y2)) in get_boundary_coordinates(level, hierarchy_info):
            grid_ax.plot([x1, x2], [y1, y2], color='white', alpha=0.5, linewidth=1)
    
    # Add grid labels for highest level
    level = RECURSION_DEPTH - 1
    for coords, area_info in hierarchy_info[level]['areas'].items():
        y_start, y_end, x_start, x_end = area_info['bounds']
        center_y = (y_start + y_end) // 2
        center_x = (x_start + x_end) // 2
        
        # Add area label
        grid_ax.text(center_x, y_start + 20, area_info['name'],
                    color='white', fontsize=10, ha='center',
                    bbox=dict(facecolor='black', alpha=0.6))
    
    # Initialize data storage
    frames = []
    stats_history = []
    metrics_history = []
    
    # Calculate which frames to save
    save_frames = list(range(0, num_frames+1, save_interval))
    if num_frames not in save_frames:
        save_frames.append(num_frames)
    
    # Add initial frame
    frames.append(grid.copy())
    initial_stats = count_population_by_area(grid, hierarchy_info)
    stats_history.append(initial_stats)
    initial_metrics = calculate_infection_metrics(initial_stats)
    metrics_history.append(initial_metrics)
    
    # Run the simulation
    print(f"Running simulation: {num_frames} frames")
    for i in tqdm(range(1, num_frames+1)):
        # Update grid
        grid = update_recursive_grid(grid, param_grid)
        
        # Save frame if in save list
        if i in save_frames:
            frames.append(grid.copy())
            frame_stats = count_population_by_area(grid, hierarchy_info)
            stats_history.append(frame_stats)
            metrics = calculate_infection_metrics(frame_stats)
            metrics_history.append(metrics)
    
    print(f"Simulation completed in {time.time() - start_time:.2f} seconds")
    print("Preparing animation...")
    
    # Animation update function
    def update(frame_idx):
        frame = min(frame_idx, len(frames)-1)
        
        # Update grid image
        grid_img.set_array(frames[frame])
        
        # Update metrics visualization
        visualize_metrics(metrics_history[frame], metrics_ax)
        
        # Update hierarchy visualization
        visualize_hierarchy(hierarchy_info, stats_history[frame], hierarchy_ax)
        
        return [grid_img]
    
    # Create animation
    anim = animation.FuncAnimation(fig, update, frames=len(frames), 
                                  interval=interval, blit=False)
    
    # Save video if requested - disabled due to compatibility issues
    if save_video:
        print("Video saving disabled - displaying animation directly")
        # Uncomment below if you have ffmpeg properly installed:
        # if not os.path.exists('results'):
        #     os.makedirs('results')
        # video_path = 'results/zombie_simulation.mp4'
        # anim.save(video_path, fps=15)
        # print(f"Video saved to {video_path}")
        # return Video(video_path)
    
    plt.tight_layout()
    print("Animation ready!")
    return HTML(anim.to_jshtml())

# ========================== RUN SIMULATION ==========================

# Set up notebook to allow animations to display
from IPython.display import display, HTML
HTML("""
<style>
.animation-container {
    width: 100%;
    height: auto;
}
</style>
""")

Initializing simulation with 2 recursion levels
Total grid dimensions: 450×450


In [ ]:

# Run simulation with parameters
# Note: Adjust num_frames and save_interval for speed vs. quality
# Set save_video=False to avoid errors with video saving
run_recursive_simulation(num_frames=75, save_interval=3, interval=150, save_video=False)

Initializing simulation...
Running simulation: 75 frames


100%|██████████| 75/75 [00:03<00:00, 24.54it/s]


Simulation completed in 4.38 seconds
Preparing animation...
Animation ready!
